## Requisição API usando Pandas

Documentação: https://brapi.dev/docs/acoes/list#search-opcional

### 1- Imports

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

### 2- Consumindo dados da API

In [ ]:
url = 'https://brapi.dev/api/quote/list?'

params = {
    'token': 'Utilize_seu_token',
    'sortBy': 'volume',
    'sortOrder': 'desc'
}
response = requests.get(url, params=params)
data = response.json()['stocks']

df = pd.DataFrame(data)

### 3- Limpando e preparando dados

In [ ]:
df.info()          # Visualização geral dos dados contidos no DF
df.isnull().sum()  # Verificação de quais colunas contem dados que estão nulos e a quantidade

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   stock       1693 non-null   object 
 1   name        1693 non-null   object 
 2   close       1693 non-null   float64
 3   change      1691 non-null   float64
 4   volume      1693 non-null   int64  
 5   market_cap  1156 non-null   float64
 6   logo        1693 non-null   object 
 7   sector      1288 non-null   object 
 8   type        1693 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 119.2+ KB
Out[85]: stock           0
name            0
close           0
change          2
volume          0
market_cap    537
logo            0
sector        405
type            0
dtype: int64

#### 3.1- Coluna Change

In [ ]:
df[df['change'].isnull()]           # Vizualizando as linhas com valores nulos da coluna 'Change'
df = df.dropna(subset=['change'])   # Removi apenas os nulos da coluna 'Change'

#### 3.2- Coluna Type

In [ ]:
# Verificando quais valores a coluna type tem
df['type'].unique()

# Gerando dicionário com os valores verificados acima e as respectivas traduções
dict_type = {'stock': 'acao', 'fund': 'fundo', 'bdr': 'bdr'}

# Sobrescrevendo o DF com os valores do dicionário
df['type'] = df['type'].map(dict_type)

#### 3.3- Coluna Sector

In [ ]:
# Rodei essa linha com os 3 tipos de papeis, e como a coluna 'sector' será importante apenas para 'Acoes' e nessa configuração não possui valores Nulos, decidi não tratá-los
df[df['sector'].isnull() & (df['type'] == 'Acao')]

# Fiz o mesmo tratamento da coluna Type
df['sector'].unique()

dict_sector = {
    'Retail Trade': 'Comércio Varejista', 'Health Services': 'Serviços de Saúde', 'Finance': 'Finanças', 'Non-Energy Minerals': 'Minerais Não Energéticos', 'Energy Minerals': 'Minerais Energéticos', 'Consumer Non-Durables': 'Bens de Consumo Não Duráveis', 'Technology Services': 'Serviços de Tecnologia', 'Commercial Services': 'Serviços Comerciais', 'Utilities': 'Serviços Públicos',
    'Transportation': 'Transporte', 'Consumer Services': 'Serviços ao Consumidor', 'Process Industries': 'Indústrias de Processo', 'Miscellaneous': 'Diversos', 'Distribution Services': 'Serviços de Distribuição', 'Producer Manufacturing': 'Manufatura de Produtores', 'Electronic Technology': 'Tecnologia Eletrônica', 'Health Technology': 'Tecnologia de Saúde', 'Communications': 'Comunicações', 'Industrial Services': 'Serviços Industriais', 'Consumer Durables': 'Bens de Consumo Duráveis'
}

df['sector'] = df['sector'].map(dict_sector)

### 4- Gerando novos Dataframes (Ações, Fundos e BDR's)
Para análises futuras pode ser interessante separar um dataframe para cada tipo de papel

In [ ]:
# ********* SE LIGA NESSA CÉLULA, FICOU MUITO TOP A LÓGICA *********

# Gerei uma lista com os valores de type ['acao', 'fundo', 'bdr']
paper_type = df['type'].unique()

# Essa função está usando o método globals() para gerar os dataframes com nome dinâmicos usando como base a lista definida acima
def group_for_type(dataframe, paper_type):

    globals()[f'df_{paper_type}'] = dataframe[dataframe['type'] == paper_type]
    return globals()[f'df_{paper_type}']

# Aqui iterei sobre cada valor da lista chamando a função e usando o valor como parâmetro
for paper in paper_type:
    group_for_type(df, paper)


"""
FORAM GERADOS 3 NOVOS DATAFRAMES:
* df_acao
* df_fundo
* df_bdr
"""

Out[89]: '\nFORAM GERADOS 3 NOVOS DATAFRAMES:\n* df_acao\n* df_fundo\n*df_bdr\n'